<a href="https://colab.research.google.com/github/williamlidberg/Geographical-Intelligence-Lab/blob/main/notebooks/Machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning on vector data

The core idea of machine learning is to program a system using data instead of with code. There are multiple types of machine learning which are used for different types of data such as images, text or numbers, but there are two main subdomains: traditional machine learning and deep learning. Machine learning was introduced around 1940 and deep learning 1963 but it was mainly deep learning that exploded in the 2000s. Deep learning is great but it is not always the best solution, especially when working with tablular data (tables with rows and columns). Therefore, this module will introduce you to traditional machine learning on geopandas dataframes.
Random forest

Random forests is a very robust machine learning method which I both love and hate. I love it because it is so easy to use and always provide a good baseline. I hate it because it is hard to come up with something more powerful and novel.

Random forest can be used for both classification and regression and works by building many decicion trees on randomly selected parts of your dataframe. Multiple deicion trees makes a decision forest hence the name random forest.

### Download wetlands
This will give us three shapefiles and a pdf describing the data. We are mainly interested in VMI_C_Objekt_KLAR_2022 and VMI_C_Vatten_TOT_2022. The first contains the wetland polygons and the second contains polygons of water bodies within each wetland. VMI_C_Objekt_KLAR_2022 also contains our target variable which is the nature value classification of each wetland. We want to build a model that can predict the nature value of wetlands based on other data.

In [ ]:
from urllib.request import urlretrieve

# Wetlands
url = ('https://ext-dokument.lansstyrelsen.se/gemensamt/geodata/ShapeExport/Lsti.Inv_vmi_objekt.zip')
filename = 'Lsti.Inv_vmi_objekt.zip'
urlretrieve(url, filename)
!unzip -o /content/Lsti.Inv_vmi_objekt.zip -d /content/wetlands
# Ditches
url = ('https://geodata.naturvardsverket.se/nedladdning/Diken/Diken_Sverige/Diken_lansvis/Diken_I.zip')
filename = '/content/Diken_I.zip'
urlretrieve(url, filename)
!unzip -o '/content/Diken_I.zip' -d /content/ditches
# Valuable forest
url = ('https://ext-dokument.lansstyrelsen.se/gemensamt/geodata/ShapeExport/Lsti.LstI_Inv_Skogliga_vardekarnor.zip')
filename = 'Lsti.LstI_Inv_Skogliga_vardekarnor.zip'
urlretrieve(url, filename)
!unzip -o /content/Lsti.LstI_Inv_Skogliga_vardekarnor.zip -d /content/valued_forest
# Water surfaces
url = ('https://opendata-download.smhi.se/svar/Vattenytor_2016.zip')
filename = '/content/Vattenytor_2016.zip'
urlretrieve(url, filename)
!unzip -o '/content/Vattenytor_2016.zip' -d /content/waterbodies

In [ ]:
import geopandas as gpd
# EPSG 3006 is the official swedish coordinate system that most swedish data will be develivred in.
wetlands = gpd.read_file('/content/wetlands/Lsti.Inv_vmi_objekt.shp', crs='EPSG:3006') # These contain the targer variable.
waterbodies = gpd.read_file('/content/waterbodies/Vattenytor_2016.shp')
waterbodies = waterbodies.to_crs('EPSG:3006')
ditches = gpd.read_file('/content/ditches/Diken_I.gpkg', crs='EPSG:3006') # Note that this is a geopackage
ditches= ditches.to_crs('EPSG:3006')
forest = gpd.read_file('/content/valued_forest/Lsti.LstI_Inv_Skogliga_vardekarnor.shp', crs='EPSG:3006')
lakes = gpd.read_file('/content/waterbodies/Vattenytor_2016.shp', crs='EPSG:3006')

In [ ]:
wetlands # Its the column Nv_klass_G that we want to predict.

Start by droping columns we dont need to make it easier to follow the process.

In [ ]:
columns_to_keep = ['OBJEKTNR', 'KLASS', 'geometry']
wetlands = wetlands[columns_to_keep] # Drop columns not in columns_to_keep
wetlands
# class_names = ['1. very high nature valye', '2. high nature value', '3. some nature value', '4. low nature value']

Intersect the ditch lines with the wetland polygons and calculate the lenght of ditch channels within each wetland.

In [ ]:
intersect = gpd.overlay(ditches, wetlands, how='intersection')
intersect['ditch_length'] = intersect.geometry.length # Calculate length of intersecting ditch lines
wetland_ditch_length = intersect.groupby('OBJEKTNR')['ditch_length'].sum().reset_index() # Aggregate lengths by wetland polygon
wetlands_with_ditch_length = wetlands.merge(wetland_ditch_length, on='OBJEKTNR', how='left').fillna({'ditch_length': 0})
wetlands_with_ditch_length

Intersect the wetland waterbodies polygons with the wetland polygons and calculate the waterbody area within each wetland.

In [ ]:
import geopandas as gpd

intersect = gpd.overlay(waterbodies, wetlands, how='intersection')
intersect['water_area'] = intersect.geometry.area # Calculate water area for intersecting waterbodies
area_waterbodies = intersect.groupby('OBJEKTNR')['water_area'].sum().reset_index()
wetlands_with_waterbodies = wetlands.merge(area_waterbodies, on='OBJEKTNR', how='left').fillna({'water_area': 0})
wetlands_with_waterbodies # notice the new column "water_area"

For lakes and valuable forests we need to calculate the distance to nearest lake instead of intersecting wetlands and lakes.

Start with lakes

In [ ]:
import geopandas as gpd
from scipy.spatial import cKDTree

wetlands_with_lake_distance = wetlands.copy()
lake_spatial_index = cKDTree(lakes.centroid.apply(lambda x: (x.x, x.y)).tolist()) # Create a spatial index for the lake centroids

# Function to calculate the nearest distance
def nearest_distance(point):
    distance, idx = lake_spatial_index.query((point.x, point.y))
    return distance

wetlands_with_lake_distance['nearest_lake_distance'] = wetlands_with_lake_distance.centroid.apply(nearest_distance) # Apply the function to each wetland centroid to get the nearest distance to a lake
wetlands_with_lake_distance

Then do the same with valuable forests

In [ ]:
import geopandas as gpd
from scipy.spatial import cKDTree

wetlands_with_forest_distance = wetlands.copy()
forest_spatial_index = cKDTree(forest.centroid.apply(lambda x: (x.x, x.y)).tolist()) # Create a spatial index for the lake centroids

# Function to calculate the nearest distance
def nearest_forest_distance(point):
    distance, idx = forest_spatial_index.query((point.x, point.y))
    return distance

wetlands_with_forest_distance['nearest_forest_distance'] = wetlands_with_forest_distance.centroid.apply(nearest_forest_distance) # Apply the function to each wetland centroid to get the nearest distance to a lake
wetlands_with_forest_distance

Finally you need to join all geodataframes. The data can be joined based on the attribute 'OBJEKTNR'. This should give you a dataframe with ditch lenght, area of lakes within each wetland, distance to nearest lake, distance to nearest valuable forest.

In [ ]:
import pandas as pd

attribute_dataframes = [wetlands_with_ditch_length, wetlands_with_waterbodies, wetlands_with_lake_distance, wetlands_with_forest_distance]
merged_wetlands = wetlands.copy()

# Merge the dataframes one by one based on the ID of each wetland
for df in attribute_dataframes:
    cols_to_use = df.columns.difference(merged_wetlands.columns)
    merged_wetlands = pd.merge(merged_wetlands, df[cols_to_use], left_index=True, right_index=True, how='outer')
merged_wetlands

The final step is to drop the column for the ID and geometry so the machine learning model does not attempt to train on those attributes.

In [ ]:
clean_data = merged_wetlands.drop(['geometry', 'OBJEKTNR'], axis=1)
clean_data = clean_data.reset_index(drop=True)
clean_data

# Machine learning with Scikit-Learn
Now when we have a dataframe with wetlands and some attributes we can use them to train a model that can predict the nature value on mires. We will use [scikit-learn](https://scikit-learn.org/stable/) to build and test a basic random forest model. To evaluate weather the model is good or not we will split the data into training 80% and testing 20%.

I highly encourage you to take some time to learn more about scikit-learn if you are interested in working with machine learning in the future. Here is a longer video if you want to learn more: [Long form video by Vincent](https://www.youtube.com/watch?v=0B5eIE_1vpU)

In [ ]:
from sklearn.model_selection import train_test_split
y = clean_data.iloc[:,0] # This is Nature value KLASS
x = clean_data.iloc[:,1:] # These are all the other attributes variables

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=0, stratify = y) # splits the data into training and testing data. test_size=0.2 means that 20% of the wetlands will be set aside for testing.


## Decision tree
It is always good to start with a simple model so we will build a [decision tree](https://scikit-learn.org/1.6/modules/generated/sklearn.tree.DecisionTreeClassifier.html) using or training data and test it on our test data.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=3) # This number determains how many decisions the tree will contain
clf.fit(x_train, y_train)


We can also inspect the trained decision tree to see whats going on.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

plt.figure(figsize=(20,10))
plot_tree(
    clf,
    feature_names=x.columns,
    class_names=[str(c) for c in clf.classes_],  # convert to strings
    filled=True,
    fontsize=8
)
plt.show()

The first decision is on the lenght of ditch channels on a wetland.

The standard way to evaluate a machine learning model is to use it to predict the test data and then compare the prediction to the test labels.

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = clf.predict(x_test) # clf is the name of our model defined above and .predict means that we use the model in prediction mode.
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

The accuracy is between 0 and 1 where 1 is 100% and something like 0.1 would be 10% accurate. In other words, the model would predict the right nature value class 10% of the time. For more detailed information we can use a classification report to see how the model preforms on different classes. The classification report produces values for precision, recall, f1-score and support. Lets break those down a bit. using class 1 (Mkt högt natuvärde/very high nature value).


*   Precision = % of how many of the predicted wetlands in class 1 that were actually in class 1.
*   Recall = % of all wetlands in class 1 that the model predicted as class 1.
*   f1-score = A combination of precision and recall. If your model has high precision but low recall, or vice versa, the F1-score will be lower. Higher number is better.
*   support = How many samples were in that class in the test data.


This might be alot to take in so focus on the f1-score for now. we want as high f1-score as possible. You can also run the prediction on the same data that it was trained on to compare that with the prediction on the test data. If the difference is big then your model has overfitted to the training data.


In [ ]:
# test the model on its own training data
from sklearn.metrics import accuracy_score
y_pred = clf.predict(x_train)
accuracy = accuracy_score(y_train, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import classification_report

y_pred_test = clf.predict(x_test)
print(classification_report(y_test, y_pred_test, zero_division=0))

Our model seems to be best at classifying wetlands with low values "Lågt naturvärde".

Another way to dive deeper into the model output is to look at a confusion matrix where each prediction is compared to the actual class of that wetland. Each square will show the number of correct predictions.


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

y_pred = clf.predict(x_test)
cm = confusion_matrix(y_test, y_pred)

# Visualize the confusion matrix
# Get the class names
class_names = ['1. very high nature valye', '2. high nature value', '3. some nature value', '4. low nature value']   # Replace with your class names

# Visualize the confusion matrix with class names
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()


## Decision forest
Now when you have seen a decision tree it's time to build a decision forest. It is the same thing but with more trees working together. The parameter in the code below "n_estimators=2" determains the number of trees in the forest. In this case it will train an ensabmle of 2 trees.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=200, max_depth=3) # create a random forest model
rf_clf.fit(x_train, y_train) # train the model

In [ ]:
y_pred = rf_clf.predict(x_test)
accuracy_score(y_test, y_pred)

We can also inspect the decision trees in a random forest but if the forest contains alot of trees this can be a bit tedious.

In [ ]:
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image

tree_index = 0  # Change this to select a different tree. Inspect the second tree in the forest by changing 0 to 1.
tree = rf_clf.estimators_[tree_index]

dot_data = export_graphviz(tree, out_file=None,
                           feature_names=x.columns,
                           class_names=[str(c) for c in clf.classes_],  # convert to strings
                           filled=True, rounded=True)

# Create a Graphviz object
graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())

With random forest we can also get an idea of what attributes are important for the model using feature importance. Keep in mind that machine learning is a bit of a black box so in order to see why a feature is important you would have to inspect all the decision trees in a random forest model.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Get feature importances from the trained model
feature_importance = rf_clf.feature_importances_
feature_names = x.columns
indices = feature_importance.argsort()[::-1]

# create a dataframe for nicer plot
importance_df = pd.DataFrame({'Feature': feature_names[indices],
                              'Importance Score': feature_importance[indices]})

# higher values are more important for the model
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance Score', y='Feature', data=importance_df, hue='Feature', palette='viridis', legend=False)
plt.title("Feature Importances")
plt.xlabel("Importance Score")
plt.ylabel("Features")
plt.show()

### Task 3
Train a random forest model with more trees and see if that improves the result. Can you think of a reason to not build as many trees as possible?


# Implement your model
You have now trained a model and evaluated the preformance of the model. The next step is to combine the prediction with the original dataframe. In this case the geodataframe with all the combined data "merged_wetlands". Note that we want to keep the geometry and ID so we can save the result as a geopackage or shapefile.

In [ ]:
import pandas as pd
subset_df = merged_wetlands[x_train.columns] # Subset the DataFrame to include only the columns used for training

predictions = rf_clf.predict(subset_df) # run the prediction
predictions_df = pd.DataFrame(predictions, columns=['Prediction']) # Convert predictions to a DataFrame

final_df = pd.concat([merged_wetlands, predictions_df], axis=1) # merge predictions with the original geodataframe
final_df.to_file('/content/predicted_wetlands.gpkg', driver='GPKG')

# Machine learning on raster data
Start by installing geopandas and cloning the course github repositroy where some example data is stored. The raster data will be stored under /content/Analyses-of-Environmental-Data-2/data/rasters and the field plots will be stored as csv files under under /content/Analyses-of-Environmental-Data-2/data/


In [ ]:
!pip install geopandas
!git clone https://github.com/williamlidberg/Analyses-of-Environmental-Data-2.git

### Import and inspect the field data csv

In [ ]:
import pandas as pd
soildata = pd.read_csv('/content/Analyses-of-Environmental-Data-2/data/Krycklan_Soilsurvey_data.csv', sep=';')
soildata

### Plot the data

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

soildata_gdf = gpd.GeoDataFrame(soildata, geometry=gpd.points_from_xy(soildata.East, soildata.North), crs=3006)
plt.rcParams["figure.figsize"] = (10,20)
soildata_gdf.plot(column='SMC', cmap='viridis_r', legend=True)

## Rasterdata

The following raster layers were calculated from the digital elevation model using whitebox tools. Don't panic, we will not use all of them.

1.   DownslopeIndex_2m
2.   DownslopeIndex_4m
3.   DepthToWater_1ha
4.   DepthToWater_2ha
5.   DepthToWater_4ha
6.   DepthToWater_8ha
7.   DepthToWater_16ha
8.   DepthToWater_32ha
9.   ElevationAboveStream_1ha
10.  ElevationAboveStream_2ha
11.  ElevationAboveStream_4ha
12.  ElevationAboveStream_8ha
13.  ElevationAboveStream_16ha
14.  ElevationAboveStream_32ha
15.  PennocLandformClassification
16.  PlanCurvature
17.  RelativeTopographicPosition
18.  TopographicWetnessIndex
19.  WILT
20.  DEM
21.  Slope
22.  DInfFlowaccumulation

You need to extract the pixel values to the field plots. This can be done using a combination of [rasterio](https://rasterio.readthedocs.io/en/latest/) and [geopandas](https://geopandas.org/en/stable/). rasterio is a python package that focuses on reading and writing raster data. Start by installing it in your environment.

In [ ]:
!pip install rasterio

Before we export anything its a good habit to inspect some of the data to make sure it looks like expected.

In [ ]:
import rasterio
from rasterio.plot import show
dem = rasterio.open('/content/Analyses-of-Environmental-Data-2/data/rasters/dem/16m.tif')
show(dem, cmap='viridis_r')
plt.show()

## Task 1
Plot some of the raster files under /content/Analyses-of-Environmental-Data-2/data/rasters/ so you get a sense of what the data is representing.

Note that some muppet has mixed lower case and upper case letters in the names and python is case sensetive. slope and Slope are not the same.

# Extract raster values to field plots
If the data looks to be in order we can use raster io to extract the raster values to our field plots. This code first finds the x and y coordinates of each field plot in the geodataframe. "coords = [(x,y) for x, y in zip(soildata_gdf.geometry.x, soildata_gdf.geometry.y)]" it then loops over each field plot and extracts the raster values. Finally it adds the extracted values to a new column in the geodataframe. "soildata_gdf['dem'] = [x[0] for x in src.sample(coords)]"

In [ ]:
import rasterio
import geopandas as gpd


coords = [(x,y) for x, y in zip(soildata_gdf.geometry.x, soildata_gdf.geometry.y)]

# Open the raster using rasterio and extract the pixel values to the geodataframe
# dem
src = rasterio.open('/content/Analyses-of-Environmental-Data-2/data/rasters/dem/16m.tif')
soildata_gdf['dem'] = [x[0] for x in src.sample(coords)] # Naming is important to keep things in order
# Slope
src = rasterio.open('/content/Analyses-of-Environmental-Data-2/data/rasters/Slope/16m.tif')
soildata_gdf['Slope'] = [x[0] for x in src.sample(coords)]
# PlanCurvature
src = rasterio.open('/content/Analyses-of-Environmental-Data-2/data/rasters/PlanCurvature/16m.tif')
soildata_gdf['PlanCurvature'] = [x[0] for x in src.sample(coords)]
# RelativeTopographicPosition
src = rasterio.open('/content/Analyses-of-Environmental-Data-2/data/rasters/RelativeTopographicPosition/16m.tif')
soildata_gdf['RelativeTopographicPosition'] = [x[0] for x in src.sample(coords)]
# TopographicWetnessIndex
src = rasterio.open('/content/Analyses-of-Environmental-Data-2/data/rasters/TopographicWetnessIndex/16m.tif')
soildata_gdf['TopographicWetnessIndex'] = [x[0] for x in src.sample(coords)]
# DownslopeIndex_2m
src = rasterio.open('/content/Analyses-of-Environmental-Data-2/data/rasters/DownslopeIndex_2m/16m.tif')
soildata_gdf['DownslopeIndex_2m'] = [x[0] for x in src.sample(coords)]
# DepthToWater_1ha
src = rasterio.open('/content/Analyses-of-Environmental-Data-2/data/rasters/DepthToWater_1ha/16m.tif')
soildata_gdf['DepthToWater_1ha'] = [x[0] for x in src.sample(coords)]
# DepthToWater_8ha
src = rasterio.open('/content/Analyses-of-Environmental-Data-2/data/rasters/DepthToWater_8ha/16m.tif')
soildata_gdf['DepthToWater_8ha'] = [x[0] for x in src.sample(coords)]


### Now you have a new geodataframe with both the field data and the raster data

In [ ]:
soildata_gdf

It can be useful to plot one of the variables to see if it makes any sense. compare this plot to the raster plots you did above.

In [ ]:
plt.rcParams["figure.figsize"] = (10,20)
soildata_gdf.plot(column='dem', cmap='viridis_r')

Since we are only interested in soil moisture now we will drop the other Y-variables. We also need to split the data into training data and testing data. The model will be trained on the training data and evaluated on the test data just like in module 7.

In [ ]:
soildata_clean = soildata_gdf.drop(soildata_gdf.columns[2:10], axis=1) # drops column 3 to 10
soildata_clean = soildata_clean.drop(soildata_gdf.columns[0], axis=1) # drops column 0 which is the text for the soil moisture
soildata_clean # SMC_code 	= Soil moisture code

# Split data into training and testing
Here we will use stratified sampling which means that sklearn will include examples of all classes in both the training data and the testing data.

In [ ]:
from sklearn.model_selection import train_test_split
y = soildata_clean.iloc[:,0] # This is soil moisture
x = soildata_clean.iloc[:,1:] # These are all the topographical variables

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=0, stratify = y)

## Tuning the hyperparameters
In the field of machine learning things are named differntly than in traditional statistics. In statistics the settings for a model is sometimes refered to as features. However, in machine learning the features are the data you extracted to the points and the setting of the model is instead called hyperparameters. Much cooler. It is quite common to fiddle with these hyper paramters to see what works and this process can be autmated. This is known as tuning the hyperparameters.

Here is an example using a tune grid where multiple models will be trained using all possible combinations of the settings listed bellow. This is a brute force approach and very demanding of your hardware. But computer time is cheaper than human time so lets do it.

In [ ]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
model = RandomForestClassifier() # note that we are using classification for the soil moisture classes


tune_grid = {'n_estimators': [50, 100, 500],
               'max_features': ['sqrt'],
               'max_depth': [4,5,6],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True]}

rf_random = RandomizedSearchCV(estimator = model, param_distributions = tune_grid, random_state=0, n_jobs = -1)

# Train the model using the optimal hyperparameters
rf_random.fit(x_train, y_train)

print('The best combination of hyperparameters was', rf_random.best_params_)

### Inspect important features

In [ ]:
import numpy as np
best_rf = rf_random.best_estimator_

# Extract feature importances
importances = best_rf.feature_importances_

# Sort from most important to least important
indices = np.argsort(importances)[::-1]

feature_names = list(x_train.columns)
importances = best_rf.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10, 8))
plt.title("Feature Importance (Random Forest)")
plt.barh(np.array(feature_names)[indices], importances[indices])
plt.xlabel("Importance")
plt.gca().invert_yaxis()   # largest at top
plt.show()

Evaluate the model just like in module 7. Note that the accuracy is between 0 and 1 so 0.5 is 50%.

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = rf_random.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Inspect the F1-score for each soil moisture class and pay attention to the support column which shows how many plots are within each category. It's harder to learn with few examples from a class. Remember that the soil moisture classes were


*   1 = Dry
*   2 = Mesic
*   3 = Mesic-moist
*   4 = Moist
*   2 = Wet

In [ ]:
from sklearn.metrics import classification_report

y_pred_test = rf_random.predict(x_test)
print(classification_report(y_test, y_pred_test, zero_division=0))

# Implement a machine learning model on raster data
Now we have a working model that we want to apply to the Krycklan catchment. We need to read all the rasterlayers, apply the model and then save the result as a new raster.

All rasterdata will be read into numpy arrays using gdal.

In [ ]:
from osgeo import gdal_array
import numpy as np
# Read raster data as numeric array from file
dem = gdal_array.LoadFile('/content/Analyses-of-Environmental-Data-2/data/rasters/dem/16m.tif')
Slope = gdal_array.LoadFile('/content/Analyses-of-Environmental-Data-2/data/rasters/Slope/16m.tif')
PlanCurvature = gdal_array.LoadFile('/content/Analyses-of-Environmental-Data-2/data/rasters/PlanCurvature/16m.tif')
RelativeTopographicPosition = gdal_array.LoadFile('/content/Analyses-of-Environmental-Data-2/data/rasters/RelativeTopographicPosition/16m.tif')
TopographicWetnessIndex = gdal_array.LoadFile('/content/Analyses-of-Environmental-Data-2/data/rasters/TopographicWetnessIndex/16m.tif')
DownslopeIndex_2m = gdal_array.LoadFile('/content/Analyses-of-Environmental-Data-2/data/rasters/DownslopeIndex_2m/16m.tif')
DepthToWater_1ha = gdal_array.LoadFile('/content/Analyses-of-Environmental-Data-2/data/rasters/DepthToWater_1ha/16m.tif')
DepthToWater_8ha = gdal_array.LoadFile('/content/Analyses-of-Environmental-Data-2/data/rasters/DepthToWater_8ha/16m.tif')


Make a list of all arrays you wish to include. Note that you need to add or remove the variables in both the list and the converted dataframe.

In [ ]:
# Make a list of all arrays. you can
list_or_all_rasters = [dem, Slope, PlanCurvature, RelativeTopographicPosition, TopographicWetnessIndex, DownslopeIndex_2m, DepthToWater_1ha, DepthToWater_8ha]

all_data = np.array(list_or_all_rasters)
all_data=all_data.reshape(8,738*662).T # The shape is from the original DEM

df_data=pd.DataFrame(all_data,columns=['dem', 'Slope', 'PlanCurvature', 'RelativeTopographicPosition', 'TopographicWetnessIndex', 'DownslopeIndex_2m', 'DepthToWater_1ha', 'DepthToWater_8ha'])


In [ ]:
result = rf_random.predict(df_data)

# Save the data as a raster file with coordinates and extent from one of the input layers
result = result.reshape(738,662)
extent = rasterio.open('/content/Analyses-of-Environmental-Data-2/data/rasters/dem/16m.tif')

with rasterio.Env():
  profile = extent.profile
  with rasterio.open('/content/Analyses-of-Environmental-Data-2/data/rasters/prediction.tif', 'w', **profile) as dst:
        dst.write(result, 1)

You can also plot the result for a quick inspection

In [ ]:
import rasterio
from matplotlib import pyplot as plt
src = rasterio.open('/content/Analyses-of-Environmental-Data-2/data/rasters/prediction.tif')
plt.imshow(src.read(1), cmap='viridis_r')
plt.show()